In [2]:
!pip install -r requirements.txt

  Using cached awswrangler-2.17.0-py3-none-any.whl (251 kB)
  Using cached Jinja2-3.1.1-py3-none-any.whl (132 kB)
  Using cached MarkupSafe-2.0.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (30 kB)
  Using cached pandas_profiling-3.1.0-py2.py3-none-any.whl (261 kB)
  Using cached requests_aws4auth-1.1.2-py2.py3-none-any.whl (24 kB)
  Using cached jsonpath_ng-1.5.3-py3-none-any.whl (29 kB)
  Using cached pyarrow-8.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)
  Using cached opensearch_py-2.0.0-py2.py3-none-any.whl (204 kB)
  Using cached redshift_connector-2.0.909-py3-none-any.whl (112 kB)
  Using cached gremlinpython-3.6.1-py2.py3-none-any.whl (73 kB)
  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached progressbar2-4.2.0-py2.py3-none-any.whl (27 kB)
  Using cached pg8000-1.29.3-py3-none-any.whl (51 kB)
  Using cached phik-0.12.2-cp38-cp38-m

In [3]:
import pandas as pd
import numpy as np
from boto3 import Session
import awswrangler as wr
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
amostra = pd.read_csv('cx_rotulos_perfis.csv')

In [5]:
len(amostra)

6661

In [6]:
amostra['dt_pesquisa'].value_counts()

2022-04-01    2491
2022-02-01    2158
2021-11-01    2012
Name: dt_pesquisa, dtype: int64

In [7]:
amostra['cpf'] = amostra['cpf'].str.replace('"', '')
amostra['cpf'] = amostra['cpf'].astype('int')

In [8]:
amostra.head()

,dt_pesquisa,cd_yearmonth,cpf,email,barrado,compartilhador,multiplicador,acumulador,controle
0,2022-04-01,202204,71152883488,Elizandras2015@gmail.com,0,1,1,1,0
1,2022-04-01,202204,3873553082,Ketlyn.schepp@gmail.com,1,0,1,0,0
2,2022-04-01,202204,16233227747,Taynacarvalhoygor@gmail.com,1,1,1,0,0
3,2022-04-01,202204,4815944504,jossieldejesusaragao@gmail.com,0,1,1,0,0
4,2022-04-01,202204,4993495390,l.noleto1313@gmail.com,0,0,1,0,0


In [9]:
amostra['mes'] = amostra['dt_pesquisa']

In [10]:
query = """

select *
from customer_sandbox_zone.customer_perfis_predictions
where cast(mes as varchar) in ('2022-04-01','2022-02-01', '2021-11-01')

"""

In [11]:
boto3_session = Session(region_name="sa-east-1")

def wr_athena_query():
    staging_path = "s3://will-prod-ml-platform-sagemaker-studio/staging/flavia-costa/"
    
    df = wr.athena.read_sql_query(
        sql=query,
        boto3_session=boto3_session
        , database='customer_sandbox_zone'
    , s3_output=staging_path)
    return df

df = wr_athena_query()

In [12]:
len(df)

4086167

In [15]:
df.head()

,cpf,mes,modelo_credito_ativo,acumulador_score,barrado_score,multiplicador_credito_ativo_score,multiplicador_credito_inativo_score
0,96388188168,2022-04-01,1,0.431571,0.112509,0.207719,0.620267
1,96388218415,2021-11-01,1,0.405964,0.137417,0.388608,0.322077
2,96388218415,2022-02-01,1,0.421761,0.108630,0.396141,0.322077
3,96388218415,2022-04-01,1,0.421761,0.108630,0.364574,0.322077
4,96388390587,2021-11-01,0,0.329152,0.370875,0.777375,0.847742


In [16]:
amostra_add = pd.merge(amostra, df, on='cpf', how='inner')


In [17]:
len(amostra_add)

11933

In [18]:
amostra_add.head()

,dt_pesquisa,cd_yearmonth,cpf,email,barrado,compartilhador,multiplicador,acumulador,controle,mes_x,mes_y,modelo_credito_ativo,acumulador_score,barrado_score,multiplicador_credito_ativo_score,multiplicador_credito_inativo_score
0,2022-04-01,202204,3873553082,Ketlyn.schepp@gmail.com,1,0,1,0,0,2022-04-01,2022-04-01,0,0.284631,0.551486,0.388578,0.393100
1,2022-04-01,202204,56664222353,ictarcisioazevedo@hotmail.com,1,1,1,0,0,2022-04-01,2022-04-01,0,0.289996,0.521491,0.442602,0.423703
2,2022-04-01,202204,12348853442,ferrerojoseaneferreiradasilva@gmail.com,1,1,1,0,0,2022-04-01,2022-02-01,0,0.285623,0.544964,0.503897,0.783629
3,2022-04-01,202204,12348853442,ferrerojoseaneferreiradasilva@gmail.com,1,1,1,0,0,2022-04-01,2022-04-01,0,0.286496,0.540601,0.558440,0.783629
4,2022-02-01,202202,5641812340,regislandiobranco@gmail.com,1,0,0,0,0,2022-02-01,2021-11-01,0,0.265780,0.628498,0.566996,0.577819


In [19]:
amostra_add['mes_x'] = pd.to_datetime(amostra_add['mes_x'], format='%Y-%m-%d')
amostra_add['mes_y'] = pd.to_datetime(amostra_add['mes_y'], format='%Y-%m-%d')

In [20]:
amostra_add = amostra_add.query('mes_x == mes_y')

In [21]:
len(amostra_add)

4646

In [22]:
amostra_add.head()

,dt_pesquisa,cd_yearmonth,cpf,email,barrado,compartilhador,multiplicador,acumulador,controle,mes_x,mes_y,modelo_credito_ativo,acumulador_score,barrado_score,multiplicador_credito_ativo_score,multiplicador_credito_inativo_score
0,2022-04-01,202204,3873553082,Ketlyn.schepp@gmail.com,1,0,1,0,0,2022-04-01,2022-04-01,0,0.284631,0.551486,0.388578,0.393100
1,2022-04-01,202204,56664222353,ictarcisioazevedo@hotmail.com,1,1,1,0,0,2022-04-01,2022-04-01,0,0.289996,0.521491,0.442602,0.423703
3,2022-04-01,202204,12348853442,ferrerojoseaneferreiradasilva@gmail.com,1,1,1,0,0,2022-04-01,2022-04-01,0,0.286496,0.540601,0.558440,0.783629
5,2022-02-01,202202,5641812340,regislandiobranco@gmail.com,1,0,0,0,0,2022-02-01,2022-02-01,0,0.285649,0.546401,0.585139,0.585018
8,2022-02-01,202202,2664392506,grasisilvadossantospassos2018@gmail.com,0,1,0,1,0,2022-02-01,2022-02-01,1,0.589727,0.013702,0.203921,0.587548


In [23]:
amostra_add['baixo_score'] = np.where((amostra_add['acumulador_score'] <= 0.5) 
                                      & (amostra_add['barrado_score'] <= 0.5) 
                                      & (amostra_add['multiplicador_credito_ativo_score'] <= 0.5) 
                                      & (amostra_add['multiplicador_credito_inativo_score'] <= 0.5), 1, 0)

In [24]:
pd.pivot_table(amostra_add, index = 'compartilhador', columns = 'baixo_score', values = 'cpf', aggfunc = lambda x: len(x.unique()))

baixo_score,0,1
compartilhador,,
0,2037,705
1,1384,520


In [25]:
!pip install statsmodels


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [26]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [27]:
formula_text = ("""
    compartilhador ~ 
    acumulador_score
    + barrado_score
    + multiplicador_credito_ativo_score
    + multiplicador_credito_inativo_score

""")


In [28]:
logit_model = smf.logit(formula = str(formula_text), data = amostra_add).fit()
print(logit_model.summary2())

Optimization terminated successfully.
         Current function value: 0.676658
         Iterations 4
                                  Results: Logit
Model:                    Logit                  Pseudo R-squared:       0.000    
Dependent Variable:       compartilhador         AIC:                    6297.5073
Date:                     2022-11-03 13:15       BIC:                    6329.7261
No. Observations:         4646                   Log-Likelihood:         -3143.8  
Df Model:                 4                      LL-Null:                -3144.4  
Df Residuals:             4641                   LLR p-value:            0.87219  
Converged:                1.0000                 Scale:                  1.0000   
No. Iterations:           4.0000                                                  
----------------------------------------------------------------------------------
                                     Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
-------------------

In [29]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

preditoras = ['acumulador_score', 'barrado_score', 'multiplicador_credito_ativo_score', 'multiplicador_credito_inativo_score']

# Fit the classifier with max_depth=3
X = np.array(amostra_add[preditoras]).reshape(1, -1)
y = np.array(amostra_add['compartilhador']).reshape(1, -1)
X.shape
    
for i in [1, 2,3,5, 10, 20, 30]:
    clf = DecisionTreeClassifier(max_depth=i, random_state=1234)
    model = clf.fit(X, y)
    text_representation = tree.export_text(clf)
    print(text_representation)


|--- class: 0

|--- class: 0

|--- class: 0

|--- class: 0

|--- class: 0

|--- class: 0

|--- class: 0



In [30]:
#loop para binarizar por percentil apenas as numéricas
for i in preditoras:
    if amostra_add[i].dtype == 'float' or amostra_add[i].dtype == 'int':
        nm_col = i + "_bin"
        amostra_add[nm_col] = pd.qcut(amostra_add[i].rank(method='first'), q=10, precision=0)

In [31]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=572b38ddbd81091a769e868c4ded50b8a307fd9943340bb7f70c9d9a74433593
  Stored in directory: /root/.cache/pip/wheels/1b/02/6c/a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [34]:
amostra_add.head()

,dt_pesquisa,cd_yearmonth,cpf,email,barrado,compartilhador,multiplicador,acumulador,controle,mes_x,...,modelo_credito_ativo,acumulador_score,barrado_score,multiplicador_credito_ativo_score,multiplicador_credito_inativo_score,baixo_score,acumulador_score_bin,barrado_score_bin,multiplicador_credito_ativo_score_bin,multiplicador_credito_inativo_score_bin
0,2022-04-01,202204,3873553082,Ketlyn.schepp@gmail.com,1,0,1,0,0,2022-04-01,...,0,0.284631,0.551486,0.388578,0.393100,0,"(466.0, 930.0]","(3717.0, 4182.0]","(1859.0, 2324.0]","(1395.0, 1859.0]"
1,2022-04-01,202204,56664222353,ictarcisioazevedo@hotmail.com,1,1,1,0,0,2022-04-01,...,0,0.289996,0.521491,0.442602,0.423703,0,"(930.0, 1395.0]","(3253.0, 3717.0]","(1859.0, 2324.0]","(1859.0, 2324.0]"
3,2022-04-01,202204,12348853442,ferrerojoseaneferreiradasilva@gmail.com,1,1,1,0,0,2022-04-01,...,0,0.286496,0.540601,0.558440,0.783629,0,"(930.0, 1395.0]","(3253.0, 3717.0]","(2788.0, 3253.0]","(3253.0, 3717.0]"
5,2022-02-01,202202,5641812340,regislandiobranco@gmail.com,1,0,0,0,0,2022-02-01,...,0,0.285649,0.546401,0.585139,0.585018,0,"(930.0, 1395.0]","(3253.0, 3717.0]","(2788.0, 3253.0]","(2324.0, 2788.0]"
8,2022-02-01,202202,2664392506,grasisilvadossantospassos2018@gmail.com,0,1,0,1,0,2022-02-01,...,1,0.589727,0.013702,0.203921,0.587548,0,"(4182.0, 4646.0]","(0.0, 466.0]","(0.0, 466.0]","(2324.0, 2788.0]"


In [35]:
amostra_add['acumulador_score_bin'].value_counts()

(0.0, 466.0]        465
(466.0, 930.0]      465
(1395.0, 1859.0]    465
(2324.0, 2788.0]    465
(3253.0, 3717.0]    465
(4182.0, 4646.0]    465
(930.0, 1395.0]     464
(1859.0, 2324.0]    464
(2788.0, 3253.0]    464
(3717.0, 4182.0]    464
Name: acumulador_score_bin, dtype: int64

In [32]:
from apyori import apriori

In [39]:
df_assoc = amostra_add[['acumulador_score_bin', 'compartilhador']]

transactions = []

for i in range(0, df_assoc.shape[0]):
    transactions.append([str(df_assoc.values[i, j]) for j in range(0, 2)])

print(transactions[0])

rules = apriori(transactions, min_support = 0.01, min_confidence = 0.10, min_lift = 1, min_length = 2)
results = list(rules)
results = pd.DataFrame(results)
results.head(50)

['(466.0, 930.0]', '0']


,items,support,ordered_statistics
0,"((0.0, 466.0])",0.100086,"[((), ((0.0, 466.0]), 0.10008609556607835, 1.0)]"
1,"((1395.0, 1859.0])",0.100086,"[((), ((1395.0, 1859.0]), 0.10008609556607835,..."
2,"((2324.0, 2788.0])",0.100086,"[((), ((2324.0, 2788.0]), 0.10008609556607835,..."
3,"((3253.0, 3717.0])",0.100086,"[((), ((3253.0, 3717.0]), 0.10008609556607835,..."
4,"((4182.0, 4646.0])",0.100086,"[((), ((4182.0, 4646.0]), 0.10008609556607835,..."
5,"((466.0, 930.0])",0.100086,"[((), ((466.0, 930.0]), 0.10008609556607835, 1..."
6,(0),0.590185,"[((), (0), 0.5901851054670685, 1.0)]"
7,(1),0.409815,"[((), (1), 0.4098148945329316, 1.0)]"
8,"(0, (0.0, 466.0])",0.059836,"[(((0.0, 466.0]), (0), 0.5978494623655914, 1.0..."
9,"(1, (1395.0, 1859.0])",0.043693,"[(((1395.0, 1859.0]), (1), 0.43655913978494626..."


In [40]:
df_assoc = amostra_add[['barrado_score_bin', 'compartilhador']]

transactions = []

for i in range(0, df_assoc.shape[0]):
    transactions.append([str(df_assoc.values[i, j]) for j in range(0, 2)])

print(transactions[0])

rules = apriori(transactions, min_support = 0.01, min_confidence = 0.10, min_lift = 1, min_length = 2)
results = list(rules)
results = pd.DataFrame(results)
results.head(50)


['(3717.0, 4182.0]', '0']


,items,support,ordered_statistics
0,"((0.0, 466.0])",0.100086,"[((), ((0.0, 466.0]), 0.10008609556607835, 1.0)]"
1,"((1395.0, 1859.0])",0.100086,"[((), ((1395.0, 1859.0]), 0.10008609556607835,..."
2,"((2324.0, 2788.0])",0.100086,"[((), ((2324.0, 2788.0]), 0.10008609556607835,..."
3,"((3253.0, 3717.0])",0.100086,"[((), ((3253.0, 3717.0]), 0.10008609556607835,..."
4,"((4182.0, 4646.0])",0.100086,"[((), ((4182.0, 4646.0]), 0.10008609556607835,..."
5,"((466.0, 930.0])",0.100086,"[((), ((466.0, 930.0]), 0.10008609556607835, 1..."
6,(0),0.590185,"[((), (0), 0.5901851054670685, 1.0)]"
7,(1),0.409815,"[((), (1), 0.4098148945329316, 1.0)]"
8,"(0, (0.0, 466.0])",0.061558,"[(((0.0, 466.0]), (0), 0.6150537634408603, 1.0..."
9,"(0, (1395.0, 1859.0])",0.061558,"[(((1395.0, 1859.0]), (0), 0.6150537634408603,..."


In [41]:
df_assoc = amostra_add[['multiplicador_credito_ativo_score_bin', 'compartilhador']]

transactions = []

for i in range(0, df_assoc.shape[0]):
    transactions.append([str(df_assoc.values[i, j]) for j in range(0, 2)])

print(transactions[0])

rules = apriori(transactions, min_support = 0.01, min_confidence = 0.10, min_lift = 1, min_length = 2)
results = list(rules)
results = pd.DataFrame(results)
results.head(50)

['(1859.0, 2324.0]', '0']


,items,support,ordered_statistics
0,"((0.0, 466.0])",0.100086,"[((), ((0.0, 466.0]), 0.10008609556607835, 1.0)]"
1,"((1395.0, 1859.0])",0.100086,"[((), ((1395.0, 1859.0]), 0.10008609556607835,..."
2,"((2324.0, 2788.0])",0.100086,"[((), ((2324.0, 2788.0]), 0.10008609556607835,..."
3,"((3253.0, 3717.0])",0.100086,"[((), ((3253.0, 3717.0]), 0.10008609556607835,..."
4,"((4182.0, 4646.0])",0.100086,"[((), ((4182.0, 4646.0]), 0.10008609556607835,..."
5,"((466.0, 930.0])",0.100086,"[((), ((466.0, 930.0]), 0.10008609556607835, 1..."
6,(0),0.590185,"[((), (0), 0.5901851054670685, 1.0)]"
7,(1),0.409815,"[((), (1), 0.4098148945329316, 1.0)]"
8,"(0, (0.0, 466.0])",0.059836,"[(((0.0, 466.0]), (0), 0.5978494623655914, 1.0..."
9,"(1, (1395.0, 1859.0])",0.046061,"[(((1395.0, 1859.0]), (1), 0.46021505376344085..."


In [44]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [45]:
x_train, x_test, y_train, y_test = train_test_split(amostra_add[preditoras]
                                                    , amostra_add['compartilhador']
                                                    , random_state=42, test_size=0.20)

In [46]:
scaler = StandardScaler()
scaler_model = scaler.fit(x_train)

x_train_scaled = scaler_model.transform(x_train)
x_test_scaled = scaler_model.transform(x_test)

In [47]:
svm_clf = svm.SVC()
svm_clf.fit(x_test_scaled, y_test)
y_predicted_svm = svm_clf.predict(x_test_scaled)
accuracy_svm = metrics.accuracy_score(y_test, y_predicted_svm).round(3)
print(accuracy_svm)

0.6


In [52]:
clf_mlp1 = MLPClassifier(solver='sgd'
                     , hidden_layer_sizes= 200
                     , max_iter = 1000
                     , random_state=1)

clf_mlp1.out_activation_ = 'softmax'

In [53]:
clf_mlp1.fit(x_test_scaled, y_test)
y_predicted_mlp1 = clf_mlp1.predict(x_test_scaled)
print(metrics.accuracy_score(y_test, y_predicted_mlp1).round(4))

0.6


In [67]:
def print_decision_rules(rf):

    for tree_idx, est in enumerate(rf.estimators_):
        tree = est.tree_
        assert tree.value.shape[1] == 1 # no support for multi-output

        print('TREE: {}'.format(tree_idx))

        iterator = enumerate(zip(tree.children_left, tree.children_right, tree.feature, tree.threshold, tree.value))
        for node_idx, data in iterator:
            left, right, feature, th, value = data

            # left: index of left child (if any)
            # right: index of right child (if any)
            # feature: index of the feature to check
            # th: the threshold to compare against
            # value: values associated with classes            

            # for classifier, value is 0 except the index of the class to return
            class_idx = np.argmax(value[0])

            if left == -1 and right == -1:
                print('{} LEAF: return class={}'.format(node_idx, class_idx))
            else:
                print('{} NODE: if feature[{}] < {} then next={} else next={}'.format(node_idx, feature, th, left, right))    


In [68]:
print_decision_rules(rf2)

TREE: 0
0 NODE: if feature[1] < -0.2868242859840393 then next=1 else next=146
1 NODE: if feature[2] < -1.513885736465454 then next=2 else next=3
2 LEAF: return class=1
3 NODE: if feature[2] < 0.8394975960254669 then next=4 else next=103
4 NODE: if feature[1] < -0.32205797731876373 then next=5 else next=102
5 NODE: if feature[2] < 0.7384240329265594 then next=6 else next=97
6 NODE: if feature[0] < -0.059588294476270676 then next=7 else next=12
7 NODE: if feature[3] < -0.5192954614758492 then next=8 else next=11
8 NODE: if feature[2] < -1.0836070775985718 then next=9 else next=10
9 LEAF: return class=0
10 LEAF: return class=1
11 LEAF: return class=1
12 NODE: if feature[2] < -0.7214855551719666 then next=13 else next=54
13 NODE: if feature[1] < -1.6320801377296448 then next=14 else next=15
14 LEAF: return class=0
15 NODE: if feature[0] < 1.704220950603485 then next=16 else next=53
16 NODE: if feature[3] < -1.1239777207374573 then next=17 else next=26
17 NODE: if feature[2] < -0.9989631175

In [73]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=42)
lr.fit(x_test_scaled, y_test)
y_predicted_lr = lr.predict(x_test_scaled)
accuracy_lr = metrics.accuracy_score(y_test, y_predicted_lr).round(4)
print(accuracy_lr)

0.6
